In [21]:
import numpy as np
import pandas as pd
# import tensorflow as tf

In [9]:
class Fingerprints():
    def __init__(self):
        self.image = []
        self.identity = []
        self.gender = []
        self.hand = []
        self.finger = []

In [25]:
# Read file and store the data in DataFrame df
# with open('images.csv', newline='') as csvfile:
#     data = list(csv.reader(csvfile))
df = pd.read_csv('images.csv')
# Store as object
fingerprints = Fingerprints()
fingerprints.image = df['Image']
fingerprints.identity = df['Identity']
fingerprints.gender = df['Gender']
fingerprints.hand = df['Hand']
fingerprints.finger = df['Finger']

['[[160 159 159 ...   0   0   0]\n [160 108 111 ...   0   0   0]\n [160 108 240 ...   0   0   0]\n ...\n [  0   0   0 ...   0   0   0]\n [  0   0   0 ...   0   0   0]\n [  0   0   0 ...   0   0   0]]']

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.drop(['Identity'],axis='columns'),df["Identity"],test_size=0.2)

In [5]:
len(X_train)

4800

In [6]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=30) # n_estimators = number of trees. Larger value => more accuracy, up to a limit
model.fit(X_train, y_train)

ValueError: could not convert string to float: '[[160 159 159 ...   0   0   0]\n [160 108 111 ...   0   0   0]\n [160 108 240 ...   0   0   0]\n ...\n [  0   0   0 ...   0   0   0]\n [  0   0   0 ...   0   0   0]\n [  0   0   0 ...   0   0   0]]'

In [ ]:
# Rerunning the generation of training set will change this value
model.score(X_test,y_test)